In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import linregress
import hvplot.pandas

# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Fetch S&P 500 monthly data for the last 60 years
sp500 = yf.download('^GSPC', period="60y", interval="1mo")

# Calculate RSI
sp500['RSI'] = calculate_rsi(sp500['Adj Close'], window=14)

# Calculate the slope of the RSI for the last 5 months in each row
N = 5  # Number of months to calculate the slope
slopes = []  # List to store slope values

for i in range(len(sp500)):
    if i < N - 1:  # Not enough data to calculate the slope
        slopes.append(np.nan)
        continue

    # Linear regression over the last N RSI values
    y = sp500['RSI'].iloc[i-N+1:i+1].values
    x = np.arange(N)
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    slopes.append(slope)

# Add slopes to the DataFrame
sp500['RSI_Slope'] = slopes

# Define buy/sell signals based on the slope
sp500['Signal'] = np.where(sp500['RSI_Slope'] > 0, 'Buy', 'Sell')

# Plot the RSI and RSI Slope with buy/sell signals
rsi_plot = sp500['RSI'].hvplot(line_color='blue', ylabel='RSI', width=800, height=400, title='S&P 500 Monthly RSI & Slope with Buy/Sell Signals (60 Years)')
buy_signals = sp500[sp500['Signal'] == 'Buy']['RSI'].hvplot.scatter(color='green', marker='^', size=100, legend=False)
sell_signals = sp500[sp500['Signal'] == 'Sell']['RSI'].hvplot.scatter(color='red', marker='v', size=100, legend=False)

# Combine plots
final_plot = rsi_plot * buy_signals * sell_signals

# Display the plot
final_plot


[*********************100%%**********************]  1 of 1 completed


:Overlay
   .Curve.RSI      :Curve   [Date]   (RSI)
   .Scatter.RSI.I  :Scatter   [Date]   (RSI)
   .Scatter.RSI.II :Scatter   [Date]   (RSI)